In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load preprocessed news data

In [11]:
path = "../Database"
usdataset = "/news_df.csv"
totalpath = path + usdataset
df = pd.read_csv(totalpath)
df['release_date'] = pd.to_datetime(df['release_date'])

# FinBERT

In [12]:
#create two columns in df: sentiment title and sentiment body
df['sentiment_title'] = np.nan
df['sentiment_body'] = np.nan
df['sentiment_body'] = df['sentiment_body'].astype(object)
df['sentiment_title'] = df['sentiment_title'].astype(object)
# df.dtypes

In [213]:
#import FinBERT
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

I did this ones for the news headlines and ones for the news bodies. Csv files sentimentheadlines_postFINBERT and bodies_sentiment_postFINBERT show the results

In [220]:
list_index = []
for i in range(len(df)):
    if i % 1000 == 0:
        print("Processed", i, "values")

    text = df['content'][i]

    try:
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        puts = F.softmax(output.logits, dim=1)
        puts = puts.detach().numpy()

        positive_score = puts[0][0]
        negative_score = puts[0][1]
        neutral_score = puts[0][2]

        sentiment_score = (positive_score,negative_score,neutral_score)
        sentiment_score = np.round(sentiment_score, 3)
        puts = np.array(sentiment_score)

        df["sentiment_body"][i]=puts
    except RuntimeError:
        print(f"Skipping article {i} due to RuntimeError")
        #create a list with all the index of the articles that are too long
        list_index.append(i)
        continue

Processed 0 values


C:\Users\cleme\AppData\Local\Temp\ipykernel_41816\3285160829.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentiment_body"][i]=puts


Skipping article 6 due to RuntimeError
Skipping article 58 due to RuntimeError
Skipping article 115 due to RuntimeError
Skipping article 139 due to RuntimeError
Skipping article 148 due to RuntimeError
Skipping article 149 due to RuntimeError
Skipping article 150 due to RuntimeError
Skipping article 151 due to RuntimeError
Skipping article 162 due to RuntimeError
Skipping article 281 due to RuntimeError
Skipping article 319 due to RuntimeError
Skipping article 340 due to RuntimeError
Skipping article 342 due to RuntimeError
Skipping article 351 due to RuntimeError
Skipping article 370 due to RuntimeError
Skipping article 372 due to RuntimeError
Skipping article 375 due to RuntimeError
Skipping article 377 due to RuntimeError
Skipping article 381 due to RuntimeError
Skipping article 387 due to RuntimeError
Skipping article 433 due to RuntimeError
Skipping article 440 due to RuntimeError
Skipping article 441 due to RuntimeError
Skipping article 458 due to RuntimeError
Skipping article 47

## Saving

# Sentiment score methods

In [ ]:
#see correlation analysis
# 1) DIFFERENCING
def diff(pos_score, neg_score, neu_score):
    return pos_score - neg_score

# 2) WEIGHTS of the sentiment score 

def weights(pos_score, neg_score, neu_score, pos_weight=0.5, neg_weight=0.5, neu_weight=0): #same as average here but see Overleaf
    final_score = pos_weight * pos_score + neg_weight * neg_score + neu_weight * neu_score
    return final_score

# 3) SIGMOID FUNCTION #has the advantage of emphasizing extreme scores and dampening the effect of neutral scores.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_official(pos_score, neg_score, neu_score):
    x = pos_score - neg_score
    final_score = sigmoid(x)
    return final_score

# 4) MAX SCORE to 1, 0 or -1
def abs_max_score(pos_score, neg_score, neu_score):
    if pos_score > neg_score and pos_score > neu_score:
        final_score = 1
    elif neg_score > pos_score and neg_score > neu_score:
        final_score = -1
    else:
        final_score = 0

    return final_score

# 5) SCORE RATIO make a def that output the highest of the three scores +pos or -neg or 0
def score_ratio(pos_score, neg_score, neu_score):
    final_score=max(pos_score, neg_score, neu_score)
    if final_score is pos_score:
        final_score= pos_score
    elif final_score is neg_score:
        final_score= -neg_score
    else:
        final_score= 0
    return final_score